In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/home-credit-default-risk/application_train.csv', index_col="SK_ID_CURR")
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

selected_cols = ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "DAYS_BIRTH", 
                 "DAYS_EMPLOYED", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", 
                 "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE", "DAYS_REGISTRATION"]
                
X, y = data[selected_cols], data['TARGET']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

Baseline model

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.784412944285295,
                           is_unbalance=True, learning_rate=0.19578329781476783,
                           min_child_samples=400, n_estimators=36, num_leaves=35,
                           reg_alpha=0.3203738316299357, reg_lambda=0.9569229543216434,
                           subsample=0.684117327613527, subsample_for_bin=180000, random_state=0)

# Training set
train_set = lgb.Dataset(X_train, label = y_train, free_raw_data=False)
val_set = lgb.Dataset(X_val, label = y_val, free_raw_data=False)

In [ ]:
from sklearn.metrics import roc_auc_score

# Train and make predicions with model
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
baseline_auc = roc_auc_score(y_val, y_pred)

print('The baseline model scores {:.5f} ROC AUC on the test set.'.format(baseline_auc))

In [ ]:
df_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv', index_col="SK_ID_CURR")
df_test = df_test[selected_cols]

preds_test = model.predict(df_test)
output_df = pd.DataFrame({"SK_ID_CURR": df_test.index,
                          "TARGET": preds_test})
output_df.to_csv("sprint1_submission.csv", index=False)